In [1]:
import pymysql
from tabulate import tabulate
import matplotlib.pyplot as plt

connection = pymysql.connect(
    user='root',
    password='root',
    host='localhost',
    database='library_app'
)
cursor = connection.cursor()


def get_existing_book_suggestions(prefix):
    query = "SELECT book_ID, BName FROM book WHERE BName LIKE %s OR CAST(book_ID AS CHAR) LIKE %s"
    cursor.execute(query, (f'{prefix}%', f'{prefix}%'))
    suggestions = cursor.fetchall()
    return suggestions

def add_book():
    print("Enter Book Details")
    try:
        prefix = input("Enter the starting characters of the book name or ID:")
        
        # Get existing book suggestions based on the entered prefix
        suggestions = get_existing_book_suggestions(prefix)

        if suggestions:
            print("Existing Book Suggestions:")
            for suggestion in suggestions:
                print(f"ID: {suggestion[0]}, Name: {suggestion[1]}")

            selected_book_id = int(input("Select an existing Book ID from the suggestions or enter a new Book ID:"))
        else:
            selected_book_id = int(input("Enter Book ID:"))

        BName = input("Enter Book Name:")
        Author = input("Enter Author Name:")
        price = int(input("Enter Book Price:"))
        copies = int(input("Enter total copies:"))

        # Set rem_copies initially equal to copies
        rem_copies = copies

        query = "INSERT INTO book VALUES (%s, %s, %s, %s, %s, %s)"
        values = (selected_book_id, BName, Author, price, copies, rem_copies)
        cursor.execute(query, values)
        connection.commit()
        print("Insertion Success")
    except pymysql.connect.Error as error:
        print("Connection Lost:", error)


def edit_book():
    print("Edit Book details")
    try:
        i_d = int(input("Enter Book_ID:"))
        query = "SELECT * FROM book WHERE book_ID = %s"
        cursor.execute(query, (i_d,))
        pre_data = cursor.fetchone()
        if pre_data:
            BName = input("Enter a new Book Name:")
            Author = input("Enter a new Book Author:")
            price = int(input("Enter a new Book Price:"))
            copies = int(input("Enter a new Total Number of copies:"))

            # Use the existing rem_copies value from the database
            rem_copies = pre_data[5]  # Assuming Rem_copies is at index 5

            # Calculate the new rem_copies based on the existing rem_copies and updated copies value
            new_rem_copies = rem_copies + copies - pre_data[4]  # Assuming copies is at index 4

            update_query = "UPDATE book SET BName = %s, Author = %s, price = %s, copies = %s, rem_copies = %s WHERE book_id = %s"
            values = (BName, Author, price, copies, new_rem_copies, i_d)

            cursor.execute(update_query, values)
            connection.commit()

            print("Successfully Updated")
        else:
            print("Not Found.")
    except ValueError:
        print("Invalid input. Please check your input.")
    except pymysql.connect.Error as error:
        print("Error in connection:", error)
def del_book():
    print("Delete Book Details")
    try:
        i_d = int(input("Enter the Book_ID to delete:"))
        query = "SELECT * FROM book WHERE book_id = %s"
        cursor.execute(query, (i_d,))
        data = cursor.fetchone()
        if data:
            del_query = "DELETE FROM book WHERE book_id = %s"
            cursor.execute(del_query, (i_d,))
            connection.commit()

            print("Successfully Deleted")
        else:
            print("Not found with your given ID.")
    except ValueError:
        print("Invalid. Enter a valid book_id.")
    except pymysql.connect.Error as error:
        print("Error in connection:", error)
        
def search_book():
    print("Search Books")
    try:
        i_d = int(input("Enter Book_ID to search details:"))
        query = "SELECT * FROM book WHERE book_id = %s"
        cursor.execute(query, (i_d,))
        result = cursor.fetchone()
        if result:
            headers = [i[0] for i in cursor.description]
            print(tabulate([result], headers=headers))
        else:
            print("Book not found with the given ID.")
    except ValueError:
        print("Invalid. Enter a valid book_id.")
    except pymysql.connect.Error as error:
        print("Error in connection:", error)

def display_books():
    
    try:
        query = "SELECT * FROM book"
        cursor.execute(query)
        results = cursor.fetchall()
        headers = [i[0] for i in cursor.description]
        print(tabulate(results, headers=headers,))
    except pymysql.connect.Error as error:
        print("Error in connection:", error)
        
        
print("Done.")



Done.
